In [1]:
import pandas as pd
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
visa_all = pd.read_csv(r"D:\_Files\UIUC 2020 Fall\IS 445 DV\Final Project\updated_US_Visa.csv")

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,4,6,9,14,15,18,19,22,23,24,25,27,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,45,46,47,49,50,51,52,53,54,56,58,59,60,61,62,63,65,66,67,68,73,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### 1. Top 10 countries, each Visa status, each year

In [3]:
def plotting_df_of_specific_year(df, year):
    visa_all_this_year = visa_all[visa_all['decision_year'] == year]
    
    visa_certified = visa_all_this_year[visa_all_this_year['case_status'] == 'Certified']
    visa_certified_expired = visa_all_this_year[visa_all_this_year['case_status'] == 'Certified-Expired']
    visa_denied = visa_all_this_year[visa_all_this_year['case_status'] == 'Denied']
    visa_withdrawn = visa_all_this_year[visa_all_this_year['case_status'] == 'Withdrawn']
    
    df_visa_certified = pd.DataFrame(visa_certified['country_of_citizenship'].dropna().value_counts().reset_index().rename(columns = {'index': 'country', 'country_of_citizenship': 'certified'}))
    df_visa_certified_expired = pd.DataFrame(visa_certified_expired['country_of_citizenship'].dropna().value_counts().reset_index().rename(columns = {'index': 'country', 'country_of_citizenship': 'certified_expired'}))
    df_visa_denied = pd.DataFrame(visa_denied['country_of_citizenship'].dropna().value_counts().reset_index().rename(columns = {'index': 'country', 'country_of_citizenship': 'denied'}))
    df_visa_withdrawn = pd.DataFrame(visa_withdrawn['country_of_citizenship'].dropna().value_counts().reset_index().rename(columns = {'index': 'country', 'country_of_citizenship': 'withdrawn'}))

    status_count_per_country = df_visa_certified.merge(df_visa_certified_expired, how = 'outer')
    status_count_per_country = status_count_per_country.merge(df_visa_denied, how = 'outer')
    status_count_per_country = status_count_per_country.merge(df_visa_withdrawn, how = 'outer')
    
    return status_count_per_country

In [4]:
def bar_chart_for_specific_year(df):
    fig, ax = plt.subplots(figsize = (15,8))

    label_country = list(df['country'].drop_duplicates()[0:10])
    certified = list(df['certified'][0:10].fillna(0))   # fillna is needed since there is no "certified" cases in 2012
    certified_expired = list(df['certified_expired'][0:10].fillna(0))
    denied = list(df['denied'][0:10].fillna(0))
    withdrawn = list(df['withdrawn'][0:10].fillna(0))

    height_for_denied = np.add(certified, certified_expired).tolist()
    height_for_withdrawn = np.add(height_for_denied, withdrawn).tolist()

    ax.bar(label_country, certified, label = 'certified')
    ax.bar(label_country, certified_expired, label = 'certified_expired', bottom = certified)
    ax.bar(label_country, denied, label = 'denied', bottom = height_for_denied)
    ax.bar(label_country, withdrawn, label = 'withdrawn', bottom = height_for_withdrawn)

    ax.set_ylim([0, 70000])
    ax.set_xlabel('Country')
    ax.set_ylabel('Number of Cases')
    ax.set_title('Top 10 Countries in Different Visa Status in each year', fontsize = 15)
    ax.legend(fontsize = 15)

In [5]:
@ipywidgets.interact(year = [2012, 2013, 2014, 2015, 2016])
def different_year(year):
    bar_chart_for_specific_year(plotting_df_of_specific_year(visa_all, year))

interactive(children=(Dropdown(description='year', options=(2012, 2013, 2014, 2015, 2016), value=2012), Output…

### 2. Employer numbers of each state presenting in an interactive US state map

In [6]:
import bqplot
import bqplot.pyplot as plt

In [7]:
df_employer_state = pd.DataFrame(visa_all['employer_state'].dropna().value_counts())
df_employer_state = df_employer_state.reset_index().rename(columns = {'index': 'Employer_state', 'employer_name': 'Counts'})
df_employer_state['Employer_state'] = df_employer_state['Employer_state'].str.upper()

In [8]:
df_employer_state = df_employer_state.groupby(['Employer_state']).sum().reset_index().sort_values(by = 'employer_state', ascending = False)

In [9]:
state_fips = pd.read_csv(r'D:\_Files\UIUC 2020 Fall\IS 445 DV\Final Project\us-state-ansi-fips.csv')
state_fips[' stusps'] = state_fips[' stusps'].str.strip()

In [10]:
df_employer_state = pd.merge(df_employer_state, state_fips, left_on = "Employer_state", right_on = " stusps")

In [11]:
us_state_map = bqplot.AlbersUSA()
color_sc = bqplot.ColorScale(scheme = "Greens")
df_employer_state = dict(zip(df_employer_state[' st'], df_employer_state['employer_state']))
map_styles_county = {'color': df_employer_state,
              'scales': {'projection': us_state_map, 'color': color_sc}}
county_fig = plt.figure()
county_map = plt.geo(map_data = 'USStatesMap', **map_styles_county)

map_tt = bqplot.Tooltip(fields = ['id', 'name', 'color'], labels = ['State Code(FIPS)', 'States', 'Number of Employers'])
county_map.tooltip = map_tt

In [12]:
county_fig

Figure(axes=[ColorAxis(scale=ColorScale(scheme='Greens'))], fig_margin={'top': 60, 'bottom': 60, 'left': 60, '…